In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns

In [ ]:
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [ ]:
nutrition_df= pd.read_csv("C:/Users/Imen/Downloads/projet nutrition/nutrition.csv",error_bad_lines= False,delimiter='\t', nrows=100000)

In [ ]:
nutrition_df.shape

In [ ]:
nutrition_df['nutriscore_grade']

In [ ]:
for idx,column in enumerate(nutrition_df.columns):
    print(idx,column)

In [ ]:
nutrition_df.dtypes

In [ ]:
nutrition_df.dtypes.value_counts()

In [ ]:
nutrition_df.isnull().sum()

In [ ]:
print(nutrition_df.columns)

In [ ]:
new_df= nutrition_df[['code', 'url', 'creator','created_t', 'last_modified_t', 'last_modified_datetime',\
                           'product_name','energy_100g', 'saturated-fat_100g',\
                           'carbohydrates_100g', 'proteins_100g', 'salt_100g',\
                           'countries', 'pnns_groups_1',\
                           'pnns_groups_2', 'states', 'states_tags', 'states_en',\
                           'energy-kcal_100g', 'nutrition-score-fr_100g', 'nutriscore_grade', 'nova_group','fiber_100g','pnns_groups_2','additives_n','sugars_100g','carbohydrates_100g','fat_100g']]

In [ ]:
new_df.shape

In [ ]:
new_df2 = new_df.head(1000000)


In [ ]:
new_df2.head()

In [ ]:
new_df2.isnull().sum()

In [ ]:
new_df2.dtypes.value_counts()

In [ ]:
new_df2['nutrition-score-fr_100g']

In [ ]:
new_df2.rename(columns = {'nutrition-score-fr_100g': 'nutrition_score_fr_100g'}) 

In [ ]:
new_df2.dtypes.value_counts().plot.pie()

In [ ]:
import plotly.express as px
sns.heatmap(new_df2.isna())

In [ ]:
pip install plotly==5.3.1

In [ ]:
columns = list(new_df2.columns)
new_df2.plot(kind='box', subplots=True, layout=(4, 4), figsize = (12, 12));

In [ ]:

import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib as plt
from matplotlib import pyplot
import matplotlib.pyplot as plt
%matplotlib inline 

In [ ]:
#Plot the Distribution plots for the features
import warnings
warnings.filterwarnings('ignore')
plt.figure(figsize=(16,5))
plt.subplot(1,2,1)
sns.distplot(new_df2['nutrition-score-fr_100g'])

In [ ]:
#Finding the Boundary Values
print("Highest allowed",new_df2['nutrition-score-fr_100g'].mean() + 3*new_df2['nutrition-score-fr_100g'].std())
print("Lowest allowed",new_df2['nutrition-score-fr_100g'].mean() - 3*new_df2['nutrition-score-fr_100g'].std())

In [ ]:
# Finding the Outliers
new_df2[(new_df2['nutrition-score-fr_100g'] > 35.840) | (new_df2['nutrition-score-fr_100g'] < -17.590)]

In [ ]:
#Trimming of Outliers
(new_df2['nutrition-score-fr_100g'] > 35.840) | (new_df2['nutrition-score-fr_100g'] < -17.590)
new_df2

In [ ]:
#Capping on Outliers
upper_limit = new_df2['nutrition-score-fr_100g'].mean() + 3*new_df2['nutrition-score-fr_100g'].std()
lower_limit = new_df2['nutrition-score-fr_100g'].mean() - 3*new_df2['nutrition-score-fr_100g'].std()

In [ ]:
#Now, apply the Capping
new_df2['nutrition-score-fr_100g'] = np.where(
    new_df2['nutrition-score-fr_100g']>upper_limit,
    upper_limit,
    np.where(
        new_df2['nutrition-score-fr_100g']<lower_limit,
        lower_limit,
        new_df2['nutrition-score-fr_100g']
    )
)

In [ ]:
new_df2['nutrition-score-fr_100g'].describe()

In [ ]:
new_df2['nutrition-score-fr_100g']

In [ ]:
new_df2

In [ ]:
new_df3=new_df2[new_df2['nutrition-score-fr_100g'].isin([1.0,2.0,3.0,4.0])]

In [ ]:
new_df3

In [ ]:
new_df3.isnull().sum()

# SQL

In [ ]:
pip install pymysql

In [ ]:
from sqlalchemy import create_engine

In [ ]:
# database connection
hostname="127.0.0.1"
dbname="nutrition"
uname="root"
pwd="root"

In [ ]:
# create SQLAlchemy engine to connect to MySQL Database
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=hostname, db=dbname, user=uname, pw=pwd))

In [ ]:
# export dataframe to sql table                                   
new_df3.to_sql('products', engine, index=False)

# faire des visualisations dynamiques

In [ ]:
#Delete product with proteins = 0
df_proteins = new_df3[['code','proteins_100g', 'fat_100g', 'sugars_100g', 'salt_100g',
                      'energy-kcal_100g', 'energy_100g','nutriscore_grade']]

In [ ]:
df_proteins

In [ ]:
df_proteins['nutriscore_grade']

In [ ]:
import plotly.graph_objects as go
columns_to_be_analyzed = ['fat_100g', 'energy-kcal_100g', 'sugars_100g', 'salt_100g']
from plotly.subplots import make_subplots
new_df3 =new_df3.groupby(by = ['nutriscore_grade']).agg({'fat_100g':'sum' ,'sugars_100g':'sum',
                                              'salt_100g':'sum', 'energy-kcal_100g':'sum' }).reset_index()

fig = make_subplots(rows = 2, cols = 3, subplot_titles=columns_to_be_analyzed)
cnt = 0
for i in range(2):
    for j in range(3):
        if cnt == 4 :
            break
        fig.add_trace(go.Bar(x = new_df2['nutriscore_grade'].to_numpy(), 
                             y = new_df2[columns_to_be_analyzed[cnt]].to_numpy()), row = i+1, col=j+1 )
        cnt+=1
fig.update_layout(  title = 'Columns Vs Number of Purchase',font=dict(
        family="Courier New, monospace",size=12,color="#7f7f7f"),showlegend=False,autosize=True,width=1200,height=800)
fig.show()

In [ ]:
df_proteins['proteins_100g'].median()

In [ ]:
df_proteins['good_proteins']= df_proteins['proteins_100g'].apply(lambda x : 0 if x < 4.61 else 1)

In [ ]:
df_proteins

In [ ]:
sns.catplot(x='proteins_100g', y='nutriscore_grade', data=df_proteins, hue="good_proteins")

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

In [ ]:
 le = preprocessing.LabelEncoder()
df_proteins["nutriscore_grade"] =le.fit_transform(df_proteins["nutriscore_grade"])

In [ ]:
# define X, y


In [ ]:
df_proteins.isnull().sum()

In [ ]:
df_proteins = df_proteins[df_proteins.fat_100g != 0]

In [ ]:
df_proteins

In [ ]:
df_proteins.fillna(0)

In [ ]:
df_proteins.dropna(0)

In [ ]:
df_proteins.dtypes.value_counts()

In [ ]:
df_proteins.astype(int)

In [ ]:
y = df_proteins["good_proteins"]
X = df_proteins.drop("good_proteins", axis=1)

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression(max_iter=1000)


In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
#j'ai volu faire une regression logistique sur 
log_model.fit(X,y)